<a href="https://colab.research.google.com/github/AfthabShiraz/QuranClassification/blob/main/Quranclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"afthabshiraz","key":"44d2ae1b3a72f0059f2b6164534fc3fe"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download ardiyanto24/quran-recitation -p /content
!unzip /content/quran-recitation.zip -d /content/quran-recitation


Dataset URL: https://www.kaggle.com/datasets/omarkhled/quran-recitations
License(s): CC0-1.0
User cancelled operation


In [ ]:
import re

def normalize_arabic(text):
    # Remove harakat (diacritics)
    harakat = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = harakat.sub('', text)
    # Remove tatweel (kashida)
    text = text.replace('ـ', '')
    # Normalize alif forms
    text = re.sub('[إأآا]', 'ا', text)
    # Normalize yaa and ta marbuta
    text = text.replace('ى', 'ي')
    text = text.replace('ة', 'ه')
    # Remove common pause marks or unusual signs if present
    text = re.sub(r'[۩۞۝]', '', text)
    # Trim spaces
    return text.strip()

ayah_index = {}

with open('/content/tanzil.txt', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('|')
        if len(parts) == 3:
            surah = int(parts[0])
            ayah = int(parts[1])
            text = parts[2]
            norm_text = normalize_arabic(text)
            ayah_index[(surah, ayah)] = norm_text

print(f"Indexed {len(ayah_index)} ayahs.")
# Peek at first few
list(ayah_index.items())[:5]



Indexed 6236 ayahs.


[((1, 1), 'بسم ٱلله ٱلرحمٰن ٱلرحيم'),
 ((1, 2), 'ٱلحمد لله رب ٱلعٰلمين'),
 ((1, 3), 'ٱلرحمٰن ٱلرحيم'),
 ((1, 4), 'مٰلك يوم ٱلدين'),
 ((1, 5), 'اياك نعبد واياك نستعين')]

In [ ]:
import glob, os

train_files = glob.glob('/content/quran-recitation/filesq/train/**/*.wav', recursive=True)
test_files = glob.glob('/content/quran-recitation/filesq/test/**/*.wav', recursive=True)

print("Train examples:", len(train_files))
print("Test examples:", len(test_files))


Train examples: 1158
Test examples: 288


In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt-get install ffmpeg


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-mdsg2bt4
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-mdsg2bt4
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=f77bee111b1fc32761a0406d9d94725516f2f2b4d586cfdae80d1f915ccc58b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-r50j7eka/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import whisper
model = whisper.load_model("medium")


100%|█████████████████████████████████████| 1.42G/1.42G [00:32<00:00, 47.7MiB/s]


In [ ]:
audio_file = "/content/quran-recitation/filesq/train/S0_1.wav"
result = model.transcribe(audio_file, language='ar')
raw_text = result['text']

In [ ]:
print(raw_text)

 قَالُواْ لَا عِلْمَ لَنَا إِنَّكَاً تَعَلَّامُ الْغُيُوبِ


In [ ]:
!pip install rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 87.7 MB/s eta 0:00:00


In [ ]:
normalized_transcript = normalize_arabic(raw_text)
print("Raw ASR:", raw_text)
print("Normalized:", normalized_transcript)



Raw ASR:  قَالُواْ لَا عِلْمَ لَنَا إِنَّكَاً تَعَلَّامُ الْغُيُوبِ
Normalized: قالوا لا علم لنا انكا تعلام الغيوب


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Build TF-IDF matrix of all ayahs
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,2))
ayah_texts = list(ayah_index.values())
tfidf_matrix = vectorizer.fit_transform(ayah_texts)

# Encode transcript and compare
query_vec = vectorizer.transform([normalized_transcript])
cosines = cosine_similarity(query_vec, tfidf_matrix).flatten()

# Get the index of the highest cosine similarity
best_idx = np.argmax(cosines)
best_score = cosines[best_idx]
predicted_surah_ayah = list(ayah_index.keys())[best_idx]

print("Best match:", predicted_surah_ayah)
print("Cosine similarity:", best_score)
print("Text:", ayah_texts[best_idx])







Best match: (5, 109)
Cosine similarity: 0.4286461460971155
Text: يوم يجمع ٱلله ٱلرسل فيقول ماذآ اجبتم ۖ قالوا۟ لا علم لنآ ۖ انك انت علٰم ٱلغيوب


In [ ]:
import requests

def get_tafseer(surah, ayah, tafseer_id=169):
    """
    tafseer_id values differ by source. 169 = Tafsir Ibn Kathir (English).
    """
    url = f"https://api.quran.com/api/v4/tafsirs/{tafseer_id}/by_ayah/{surah}:{ayah}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        text = data['tafsir']['text']
        return text
    else:
        return "Tafseer not available."


Tafseer for 2:255

<h2>The Virtue of Ayat Al-Kursi</h2><p>This is Ayat Al-Kursi and tremendous virtues have been associated with it, for the authentic Hadith describes it as `the greatest Ayah in the Book of Allah.' Imam Ahmad recorded that `Ubayy bin Ka`b said that the Prophet asked him about the greatest Ayah in the Book of Allah, and `Ubayy answered, "Allah and His Messenger know better." When the Prophet repeated his question several times, `Ubayy said, "Ayat Al-Kursi." The Prophet commented,</p><p>«لِيَهْنِكَ الْعِلْمُ أَبَا الْمُنْذِرِ، وَالَّذِي نَفْسِي بِيَدِهِ، إِنَّ لَهَا لِسَانًا وَشَفَتَيْنِ، تُقَدِّسُ الْمَلِكَ عِنْدَ سَاقِ الْعَرْش»</p><p>(Congratulations for having knowledge, O Abu Al-Mundhir! By He in Whose Hand is my soul! This Ayah has a tongue and two lips with which she praises the King (Allah) next to the leg of the Throne.)</p><p>This Hadith was also collected by Muslim, but he did not include the part that starts with, "By He in Whose Hand..."</p><p>Imam Ahmad re

In [ ]:
surah, ayah = predicted_surah_ayah

# Fetch tafseer
tafsir_text = get_tafseer(surah, ayah)

print(f"Predicted Ayah: Surah {surah}, Ayah {ayah}")
print("Tafseer:", tafsir_text)

Predicted Ayah: Surah 5, Ayah 109
Tafseer: <h2>The Messengers ﷺ Will be Asked About Their Nations</h2><p>Allah states that on the Day of Resurrection, He will ask the Messengers about how their nations, to whom He sent them, answered and responded to their teachings. Allah said in other Ayat,</p><p>فَلَنَسْـَلَنَّ الَّذِينَ أُرْسِلَ إِلَيْهِمْ وَلَنَسْـَلَنَّ الْمُرْسَلِينَ</p><p>(Then surely, We shall question those (people) to whom it (the Book) was sent and verily, We shall question the Messengers.) 7:6, and,</p><p>فَوَرَبِّكَ لَنَسْـَلَنَّهُمْ أَجْمَعِينَ - عَمَّا كَانُواْ يَعْمَلُونَ</p><p>(So, by your Lord, We shall certainly call all of them to account. For all that they used to do.) 15:92-93. The statement of the Messengers here,</p><p>لاَ عِلْمَ لَنَآ</p><p>(We have no knowledge) is the result of the horror of that Day, according to Mujahid, Al-Hasan Al-Basri and As-Suddi. `Abdur-Razzaq narrated that Ath-Thawri said that Al-A`mash said that Mujahid said about the Ayah,</p><p>ي